In [1]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer 
import random
import io
import string
from sklearn.metrics.pairwise import cosine_similarity
import MeCab 
import numpy as np

In [4]:
mecab = MeCab.Tagger('-Owakati')

p = Path("/home/ifte/amiebot_project/amie-HelpBot/mobicontrol_data/corpus_mobicontrol.csv")
df = pd.read_csv(p, header=0, usecols=["page", "text"])

In [5]:
# Keyword Matching
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

def greeting(sentence):
    """If user's input is a greeting, return a greeting response"""
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [6]:
import re

def clean_text(text):
    replaced_text = re.sub(r'[【】]', ' ', text)       # 【】の除去
    replaced_text = re.sub(r'[・_]', '', replaced_text)       # ・ の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去
    replaced_text = re.sub(r'　', ' ', replaced_text)  # 全角空白の除去
    replaced_text = re.sub(r'\d+', '', replaced_text) # 数字の除去
    replaced_text = re.sub(r'[-/。,、.=]', ' ', replaced_text)
    return replaced_text

In [7]:
stop_words_ja = []
STOPWORD_FILE = Path("/home/ifte/amiebot_project/amie-HelpBot/amie_helpbot/" + '/assets/learning/stopword_ja.txt')
with open(STOPWORD_FILE, encoding='utf-8') as fr:
    stop_words_ja = fr.read().splitlines()

In [8]:
y = df[["page"]]
X = df["text"].apply(lambda x: mecab.parse(x).strip("\n"))
df['parsed'] = X 

In [9]:
classes = df.page.unique()
c_size = 100

result = pd.DataFrame()

for i in list(classes):
    arr_index = df[df.page == i].parsed.values
    data = ''.join(list(arr_index))
    
    all_data = clean_text(data)
    
    arr_words = np.array(all_data.split())
    #print(arr_words) 
    
    num = arr_words.shape[0]//c_size
    full = c_size * num
    rest = arr_words.shape[0] - full
    
    pad = np.zeros([c_size-rest], dtype=int)
    sc = np.concatenate((arr_words,pad))
    features = sc.reshape(num+1, c_size)
    
    df_v = pd.DataFrame(features)
    df_v["cls"] = i
    
    result = pd.concat([result, df_v]) 

    
#print(result)
#result.to_csv('processed.csv')

In [10]:
Data_cols = result.iloc[:,:-1]
tog = Data_cols.apply(lambda x: ' '.join(x), axis=1)
tog.to_csv("tog.csv")
sent = list(tog.values)

/home/ifte/amiebot_project/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words=stop_words_ja, max_df=0.95)
vectorizer.fit(sent)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.95, max_features=None,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた', 'あれ',
                            'いくつ', 'いつ', 'いま', 'いや', 'いろいろ', 'うち', 'おおまか',
                            'おまえ', 'おれ', 'がい', 'かく', 'かたち', 'かやの', 'から', 'がら',
                            'きた', 'くせ', 'ここ', 'こっち', 'こと', 'ごと', 'こちら', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [15]:
# Read a question from the user
question = [input('Please enter a question: \n')]
question = vectorizer.transform(question)

# Rank all the questions using cosine similarity to the input question
rank = cosine_similarity(question, vectorizer.transform(sent))


Please enter a question: 
mobicontrol database


In [16]:
# Grab the top 5
top = np.argsort(rank, axis=-1).T[-5:]
#print(top)

# Print top 5
for item in range(len(top)):
    idx = top[item,0]
    
    # if the dataset file structure "Quesition | Answer" then use following line  
    #print(data['Answer'].iloc[item].values[0])
    print("PageID: ",result.iloc[idx,-1],"idx",idx,"=",tog.values[idx])
    print("\n ########## \n")

PageID:  24 idx 575 = 入力 欄 に MobiControl サーバ の ドメイン に ssp を つけ て 入力 し ます 仮に MobiControl サーバ の URL が https : mobicontrol nippon co jp MobiControl だっ た と し ます https : mobicontrol nippon co jp ssp と 入力 し ます MobiControl の 最初 の インストール が V 以前 の 場合 ブラウザ の URL 入力 欄 に MobiControl サーバ の URL に サブフォルダ m を つけ て 入力 し ます 仮に MobiControl サーバ の URL が https : mobicontrol nippon co jp MobiControl だっ た と し ます https : mobicontrol nippon co jp MobiControl m と

 ########## 

PageID:  24 idx 566 = の ドメイン に ssp を つけ て 入力 し ます 仮に MobiControl サーバ の URL が https : mobicontrol nippon co jp MobiControl だっ た と し ます https : mobicontrol nippon co jp ssp と 入力 し ます MobiControl の 最初 の インストール が V 以前 の 場合 ブラウザ の URL 入力 欄 に MobiControl サーバ の URL に サブフォルダ m を つけ て 入力 し ます 仮に MobiControl サーバ の URL が https : mobicontrol nippon co jp MobiControl だっ た と し ます https : mobicontrol nippon co jp MobiControl m と 入力 し ます 現在 MobiControl

 ########## 

PageID:  324 idx 6728 = ファイル として の 指定 解除 Archive ファイル は 前回 の バックアップ 以降 に 変更 さ れ た ファイル a

In [97]:
result.loc[(result['cls'] == result.iloc[idx,-1])].values

array([['SDK', 'for', 'iOS', ..., 'の', '表示', 261],
       ['フォント', 'の', '管理', ..., 'リモート', '表示', 261],
       ['など', 'メッセージ', 'を', ..., 'MobiControl', 'で', 261],
       ...,
       ['MobiControl', 'サーバ', 'に', ..., 'jp', 'DS', 261],
       ['MC', 'ROOT', 'CERTS', ..., 'サーバ', 'の', 261],
       ['サイト', '名', 'を', ..., '0', '0', 261]], dtype=object)